In [ ]:
import ccxt
import time
from itertools import permutations
from ccxt.base.errors import RateLimitExceeded

# Replace with your actual Bybit API keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})


In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.1 / 100  # 0.1% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
TRADING_FEE = 0.001  # 0.1% trading fee per trade
STARTING_CAPITAL = 100  # Configurable capital in USDT

def get_tradable_tokens():
    """Fetches tradable tokens and caches market data to reduce API calls."""
    try:
        markets = bybit.load_markets()
        market_data_cache = {}  # Cache market data
        tradable_tokens = []
        
        for symbol, market in markets.items():
            if '/USDT' in symbol and ':USDT' not in symbol:
                if symbol not in market_data_cache:
                    try:
                        market_data_cache[symbol] = bybit.fetch_ticker(symbol)
                    except RateLimitExceeded:
                        print("⚠️ Rate limit hit while fetching ticker. Pausing...")
                        time.sleep(60)
                        continue

                volume = market_data_cache[symbol].get('quoteVolume', 0)
                if volume and volume >= MIN_VOLUME:
                    tradable_tokens.append(symbol)

        print(f"✅ Found {len(tradable_tokens)} tradable tokens")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens: {e}")
        return []

def get_price(symbol):
    """Fetch latest price for a given token if the market exists."""
    try:
        if symbol in bybit.load_markets():
            ticker = bybit.fetch_ticker(symbol)
            return float(ticker.get("last", 0)) if ticker else None
        return None
    except RateLimitExceeded:
        print("⚠️ Rate limit hit while fetching price. Pausing...")
        time.sleep(60)
        return None
    except Exception as e:
        print(f"❌ Error fetching price for {symbol}: {e}")
        return None

def log_arbitrage(trade_path, expected_return, net_profit):
    """Logs detected arbitrage trades to a file."""
    with open("triangular_arbitrage.log", "a") as log_file:
        log_file.write(f"{trade_path} | Expected: {expected_return:.4%} | Net Profit: {net_profit:.4%}\n")

def check_triangular_arbitrage(tradable_tokens):
    """Checks for triangular arbitrage opportunities within Bybit."""
    for route in permutations(tradable_tokens, 3):  # Ensure three different assets
        base_asset = route[0].split('/')[0]  # First asset in the path
        mid_asset = route[1].split('/')[0]
        final_asset = route[2].split('/')[1]  # Final conversion back to USDT

        if base_asset == final_asset:  # Ensure a valid cycle
            try:
                # Get price data for each leg
                first_leg_price = get_price(route[0])
                second_leg_price = get_price(route[1])
                third_leg_price = get_price(route[2])

                # Ensure all prices are valid
                if None in [first_leg_price, second_leg_price, third_leg_price]:
                    continue  

                # Convert using proper exchange path
                amount_after_first_trade = (STARTING_CAPITAL / first_leg_price) * second_leg_price
                amount_after_second_trade = (amount_after_first_trade / second_leg_price) * third_leg_price
                final_amount = amount_after_second_trade

                expected_return = final_amount / STARTING_CAPITAL
                net_expected_return = expected_return * (1 - TRADING_FEE) ** 3  # Adjust for fees

                # Print all paths for debugging
                print(f"🔍 Checking Arbitrage Path: {route[0]} ➝ {route[1]} ➝ {route[2]}")
                print(f"    ✅ Expected Return: {expected_return:.4%}")
                print(f"    ✅ Net Profit After Fees: {net_expected_return:.4%}")

                # Detect and log only profitable trades
                if net_expected_return > 1.001:  # Ensure profit > 0.1%
                    print(f"✅ VALID Triangular Arbitrage Detected on Bybit")
                    print(f"    ✅ Trade Path: {route[0]} ➝ {route[1]} ➝ {route[2]}")
                    print(f"    ✅ Expected Return: {expected_return:.4%}")
                    print(f"    ✅ Net Profit After Fees: {net_expected_return:.4%}")
                    print("-" * 50)
                    log_arbitrage(f"{route[0]} ➝ {route[1]} ➝ {route[2]}", expected_return, net_expected_return)
            except Exception as e:
                print(f"⚠️ Error in arbitrage check: {e}")
                continue

while True:
    try:
        bybit_tokens = get_tradable_tokens()
        if not bybit_tokens:
            print("❌ No tradable tokens found. Retrying in 10 seconds...")
            time.sleep(10)
            continue

        print("🔄 Checking for arbitrage opportunities...")

        check_triangular_arbitrage(bybit_tokens)

        print("✅ Completed this cycle. Waiting for next check...")
        time.sleep(CHECK_INTERVAL)
    except Exception as e:
        print(f"❌ Error in main loop: {e}")
        time.sleep(10)  # Avoid infinite crash loops
